<a href="https://colab.research.google.com/github/KorostinD/Diploma_project/blob/main/Denoising_autoencoder_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import librosa as lb
import librosa.display
import torch
from torch import nn
import torchvision.transforms as tfs
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os

data=np.zeros((120,89175,2))

for i in range(120):
  sample,sr=lb.load(f'/content/drive/MyDrive/audiosample_dataset/noisy_samples/{i}.wav',sr=44100)
  label,sr=lb.load(f'/content/drive/MyDrive/audiosample_dataset/clean_samples/{i}.wav',sr=44100)

  fourier_noisy= librosa.stft(sample) 
  fourierdb_noisy = librosa.amplitude_to_db(abs(fourier_noisy))
  fourierdb_noisy=fourierdb_noisy.reshape(1,89175)
  fourier= librosa.stft(label) 
  fourierdb = librosa.amplitude_to_db(abs(fourier))
  fourierdb=fourierdb.reshape(1,89175)
  for j in range(89175):
    data[i,j,0]=fourierdb_noisy[0,j]
    data[i,j,1]=fourierdb[0,j]
print(data)

[[[ 3.35150146e+01  3.51826553e+01]
  [ 2.52606163e+01  2.94613190e+01]
  [ 1.81784515e+01  6.71023321e+00]
  ...
  [ 1.82281208e+01 -2.42391396e+01]
  [ 1.89458866e+01 -3.10001802e+00]
  [ 2.23684998e+01  6.58796263e+00]]

 [[ 3.29084244e+01  3.28700485e+01]
  [ 2.67241325e+01  2.68517418e+01]
  [ 2.25874567e+00 -2.37543297e+01]
  ...
  [-1.64747391e+01 -2.37543297e+01]
  [ 8.67007637e+00 -8.44724178e+00]
  [ 1.63287926e+01  9.20096189e-02]]

 [[ 1.29019985e+01  3.25209122e+01]
  [ 2.31068573e+01  2.64932461e+01]
  [ 2.41517353e+01 -2.46485748e+01]
  ...
  [-1.71820965e+01 -2.46485748e+01]
  [ 1.93803139e+01 -8.12755203e+00]
  [ 2.80721626e+01  9.58276074e-03]]

 ...

 [[ 2.89417038e+01  2.76040363e+01]
  [ 2.36081581e+01  2.15914440e+01]
  [ 1.26234293e+01 -1.63646793e+01]
  ...
  [ 2.37872467e+01 -1.21225405e+01]
  [ 1.81007462e+01 -8.14973831e+00]
  [ 1.86087761e+01 -1.56539381e-01]]

 [[ 2.38426552e+01  3.10890732e+01]
  [ 7.85463905e+00  2.51814575e+01]
  [ 2.01278706e+01 -1.9152

In [6]:
data_tensor=torch.from_numpy(data)
data_tensor=torch.nn.functional.normalize(data_tensor,2)
print(data_tensor)

class SpectrogramDataset(Dataset):

    def __init__(self,file_label_ds):
        self.ds= file_label_ds
    def __getitem__(self, index):
        file=self.ds[index,-1,0]
        label=self.ds[index,-1,1]
        return file, label
    def __len__(self):
        return len(self.ds)

dataset=SpectrogramDataset(data_tensor)

dataloader=DataLoader(dataset,batch_size=20)

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(89175, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 128),
            nn.ReLU(True),
            nn.Linear(128, 16),
            nn.ReLU(True),
            
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 128),
            nn.ReLU(True),
            nn.Linear(128, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 89175),
            nn.Tanh()
        )

    def forward(self, x):
        x=x.to(torch.float32)
        x = self.encoder(x)
        x = self.decoder(x)
        x=x.to(torch.float64)
        return x

model=autoencoder()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,
                             weight_decay=1e-5)

num_epochs=2

spector=np.zeros(89175)
spector=torch.from_numpy(spector)
label=np.zeros(89175)
label=torch.from_numpy(label)

for epoch in range(num_epochs):
    for i in range(100):
        for j in range(89175):
          
          
          spector[j]=data_tensor[i,j,0]
          label[j]=data_tensor[i,j,1]
        # ===================forward=====================
        output = model(spector)
        loss = criterion(output, label)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # ===================log========================
        print('epoch [{}/{}], loss:{:.4f}'
            .format(epoch+1, num_epochs, loss))
        print('i='+str(i)+'\n'+str(output)+'\n -----------------------------------------------')

tensor([[[ 8.1769e-03,  9.0413e-03],
         [ 6.1630e-03,  7.5711e-03],
         [ 4.4351e-03,  1.7244e-03],
         ...,
         [ 4.4472e-03, -6.2290e-03],
         [ 4.6224e-03, -7.9665e-04],
         [ 5.4574e-03,  1.6930e-03]],

        [[ 9.2665e-03,  8.4499e-03],
         [ 7.5251e-03,  6.9028e-03],
         [ 6.3603e-04, -6.1066e-03],
         ...,
         [-4.6390e-03, -6.1066e-03],
         [ 2.4414e-03, -2.1715e-03],
         [ 4.5979e-03,  2.3653e-05]],

        [[ 1.9886e-03,  5.2986e-03],
         [ 3.5614e-03,  4.3165e-03],
         [ 3.7225e-03, -4.0160e-03],
         ...,
         [-2.6483e-03, -4.0160e-03],
         [ 2.9871e-03, -1.3242e-03],
         [ 4.3267e-03,  1.5613e-06]],

        ...,

        [[ 5.6668e-03,  7.1600e-03],
         [ 4.6225e-03,  5.6004e-03],
         [ 2.4717e-03, -4.2447e-03],
         ...,
         [ 4.6576e-03, -3.1444e-03],
         [ 3.5442e-03, -2.1139e-03],
         [ 3.6436e-03, -4.0603e-05]],

        [[ 4.2141e-03,  9.5306e-03

In [7]:
for i in range(100,120):
  for j in range(89175):
          
          
          spector[j]=data_tensor[i,j,0]
          label[j]=data_tensor[i,j,1]
  output = model(spector)
  loss = criterion(output, label)
  print('I:'+str(i-100)+'Loss:'+str(loss))

I:0Loss:tensor(1.2778e-05, dtype=torch.float64, grad_fn=<MseLossBackward0>)
I:1Loss:tensor(1.5687e-05, dtype=torch.float64, grad_fn=<MseLossBackward0>)
I:2Loss:tensor(9.8560e-06, dtype=torch.float64, grad_fn=<MseLossBackward0>)
I:3Loss:tensor(1.2590e-05, dtype=torch.float64, grad_fn=<MseLossBackward0>)
I:4Loss:tensor(9.6213e-06, dtype=torch.float64, grad_fn=<MseLossBackward0>)
I:5Loss:tensor(1.4116e-05, dtype=torch.float64, grad_fn=<MseLossBackward0>)
I:6Loss:tensor(1.2267e-05, dtype=torch.float64, grad_fn=<MseLossBackward0>)
I:7Loss:tensor(1.1650e-05, dtype=torch.float64, grad_fn=<MseLossBackward0>)
I:8Loss:tensor(1.1868e-05, dtype=torch.float64, grad_fn=<MseLossBackward0>)
I:9Loss:tensor(1.0350e-05, dtype=torch.float64, grad_fn=<MseLossBackward0>)
I:10Loss:tensor(1.0991e-05, dtype=torch.float64, grad_fn=<MseLossBackward0>)
I:11Loss:tensor(1.1247e-05, dtype=torch.float64, grad_fn=<MseLossBackward0>)
I:12Loss:tensor(8.4946e-06, dtype=torch.float64, grad_fn=<MseLossBackward0>)
I:13Loss: